# Understanding Runable

Runnable is the framework that controls the piping of invokes. 
langchain_core.runnables.base.RunnableParallel runs in parallel
langchain_core.runnables.base.RunnableSequence runs in sequence
RunnableLambda is a lambda function to wrap functions in.

## Improt Environement Variables from File
Need to have a .env file with OPENAI_API_KEY and a LANGSMITH_API_KEY

In [ ]:
from environs import Env
import os

from langchain_core.runnables import RunnableLambda,RunnableParallel




env = Env()
env.read_env("/Users/geoffreysmalling/development/langchain/.env")

# simple example



In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model
chain.invoke("cycling")

In [ ]:
from langchain_core.runnables import RunnableLambda

def add_one(x: int) -> int:
    return x + 1

def mul_two(x: int) -> int:
    return x * 2

def mul_three(x: int) -> int:
    return x * 3

runnable_1 = RunnableLambda(add_one)
runnable_2 = RunnableLambda(mul_two)
runnable_3 = RunnableLambda(mul_three)



In [ ]:
runnable_parallel_2_3 = RunnableParallel({"mul_two": runnable_2,
                                          "mul_three": runnable_3}) 

sequence = runnable_1 |runnable_parallel_2_3

print(runnable_1.invoke(1))

print(sequence.invoke(1))
await sequence.ainvoke(1)

print(sequence.batch([1, 2, 3]))
await sequence.abatch([1, 2, 3])


In [ ]:
sequence = runnable_1 | RunnableParallel(
    mul_two=runnable_2,
    mul_three=runnable_3,
)

sequence.invoke(1)
await sequence.ainvoke(1)

sequence.batch([1, 2, 3])
await sequence.abatch([1, 2, 3])

In [ ]:
sequence = runnable_1 | {  # this dict is coerced to a RunnableParallel
    "mul_two": runnable_2,
    "mul_three": runnable_3,
}

sequence.invoke(1)
await sequence.ainvoke(1)

sequence.batch([1, 2, 3])
await sequence.abatch([1, 2, 3])

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
joke_chain = (
    ChatPromptTemplate.from_template("tell me a joke about {topic}")
    | model
)
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}")
    | model
)

runnable = RunnableParallel(joke=joke_chain, poem=poem_chain)

# Display stream
output = {key: "" for key, _ in runnable.output_schema()}
for chunk in runnable.stream({"topic": "bear"}):
    for key in chunk:
        output[key] = output[key] + chunk[key].content
    print(output)  # noqa: T201

# RunnableRunnablePassthrough 

Runnable to passthrough inputs unchanged or with additional keys.

This runnable behaves almost like the identity function, except that it can be configured to add additional keys to the output, if the input is a dict.

The examples below demonstrate this Runnable works using a few simple chains. The chains rely on simple lambdas to make the examples easy to execute and experiment with.


In [ ]:
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

runnable = RunnableParallel(
    origin=RunnablePassthrough(),
    modified=lambda x: x+1
)

runnable.invoke(1) # {'origin': 1, 'modified': 2}


def fake_llm(prompt: str) -> str: # Fake LLM for the example
    return "completion"

chain = RunnableLambda(fake_llm) | {
    'original': RunnablePassthrough(), # Original LLM output
    'parsed': lambda text: text[::-1] # Parsing logic
}

chain.invoke('hello') # {'original': 'completion', 'parsed': 'noitelpmoc'}

## adding metadata

In [ ]:
from langchain_core.runnables import RunnablePassthrough

def fake_llm(prompt: str) -> str: # Fake LLM for the example
    return prompt

runnable = {
    'llm1':  fake_llm,
    'llm2':  fake_llm,
} | RunnablePassthrough.assign(
    total_chars=lambda inputs: len(inputs['llm1'] + inputs['llm2'])
)

runnable.invoke('hello')
# {'llm1': 'completion', 'llm2': 'completion', 'total_chars': 20}

In [ ]:
# Requires:
# pip install langchain docarray tiktoken

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125",api_key=env.str("OPENAI_API_KEY"))

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

print(retriever.invoke("where did harrison work?"))

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser
print(setup_and_retrieval.invoke("where did harrison work?"))

chain.invoke("where did harrison work?")